In [1]:
import os 
import pandas as pd 
import sys 
import re 

current_path = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_path,'..','..'))  # '..','..','..'
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)
    
from pipeline.plotting.TS_analysis import plot_TS
from jupyter_ipynb.NetMob_training_analysis.plotting import plot_boxplot_on_metric
from experiences.convert_df_to_latex import dataframe_to_latex,parse_index_exp1_2,parse_index_exp4
from experiences.convert_df_to_latex import update_df_metrics,load_csv
from experiences.convert_df_to_latex import tackle_trial_j as tackle_trial_for_distrib
from experiences.common_results import dic_exp_to_names,find_baseline,dic_trials

folder_path = '../../save/K_fold_validation/training_wo_HP_tuning'  # '../../save'


In [2]:
for exp_i,trials in dic_trials.items():
    if exp_i == 'set_experience_HERE' :#'Exp1_subway_in': #'set_experience_HERE': # set_experience_HERE # 'Exp1_subway_in'# 'Exp4_15min' # 'Exp4' # 'Exp3_bike_15min_h4' # 'Exp3' # 'Exp1'
        for trial_j in trials:
            df_j_all = pd.DataFrame()
            for n_bis in range(1,6):
                file_path = f"{folder_path}/{exp_i}/{dic_exp_to_names[exp_i]}/{trial_j}_bis{n_bis}/Losses_{trial_j}_bis{n_bis}.csv"
                df_j = pd.read_csv(file_path,index_col = 0)
                df_j.columns = [f'Train_{n_bis}',f'Valid_{n_bis}']
                df_j_all = pd.concat([df_j_all,df_j],axis=1)
            plot_TS(df_j_all,width=1500,height=400,bool_show=True,title=f"{exp_i}, {trial_j}",x_datetime = False)


In [2]:
import os 
import pandas as pd 
from pipeline.plotting.TS_analysis import plot_TS

def tackle_trial_j(folder_path,dic_exp_to_names,L_metrics,exp_i,trial_j,metrics,plot_losses):
    df_j_all = pd.DataFrame()
    metric_i = []
    for n_bis in range(1,6):
        df_j_all, metric_i = load_csv(folder_path,dic_exp_to_names,exp_i,trial_j,n_bis,df_j_all,metric_i,metrics)

    metric_i = pd.DataFrame(pd.DataFrame(metric_i).agg(['mean','std']).unstack()).T
    metric_i.index = [f"{trial_j}"]
    L_metrics.append(metric_i)
    if plot_losses:
            plot_TS(df_j_all,width=1500,height=400,bool_show=True,title=f"{exp_i}, {trial_j}",x_datetime = False)
    return L_metrics

folder_path = 'save/K_fold_validation/training_wo_HP_tuning' # ../../
metrics = ['rmse','mae','mase']
plot_losses = False 
experiences = ['Exp1_subway_in','Exp1_subway_out','Exp1']
for exp_i in experiences:
    trials = dic_trials[exp_i]
    print(f"\n------------------ {exp_i} ---")
    
    L_metrics = []
    for trial_j in trials:
        L_metrics = tackle_trial_j(folder_path,dic_exp_to_names,L_metrics,exp_i,trial_j,metrics,plot_losses)

    df_metrics_all = pd.concat(L_metrics)

    horizons = list(set([c[0].split('_')[-1][1:] for c in df_metrics_all.columns]))
    for horizon in horizons:
        df_horizon = df_metrics_all[[c for c in df_metrics_all.columns if c[0].endswith(f"_h{horizon}")]].dropna()

        globals()[f"df_metrics_{exp_i}_h{horizon}"] = df_horizon.sort_values(by=[(f'rmse_h{horizon}','mean')])

        baseline_index = find_baseline(exp_i,h=horizon)
        for c in globals()[f"df_metrics_{exp_i}_h{horizon}"].columns:
            if 'mean' == c[1]:
                globals()[f"df_metrics_{exp_i}_h{horizon}"][c] = globals()[f"df_metrics_{exp_i}_h{horizon}"][c].apply(lambda x: f"{round(x,2)} ({round(100*(-1+x/globals()[f'df_metrics_{exp_i}_h{horizon}'].loc[baseline_index,c]),2)}\%)")
        
        display(globals()[f"df_metrics_{exp_i}_h{horizon}"])



------------------ Exp1_subway_in ---


rmse_h1            \
                                                               mean       std   
STAEformer_subway_in_subway_out_calendar_early_...  35.48 (-0.33\%)  0.135227   
STAEformer_subway_in_subway_out_calendar_late_f...  35.58 (-0.04\%)  0.059006   
STAEformer_subway_in_calendar__e80_h1                  35.6 (0.0\%)  0.228636   
STAEformer_subway_in_subway_out_calendar_early_...   35.62 (0.08\%)  0.178211   
STAEformer_subway_in_subway_out_calendar_late_f...   35.65 (0.15\%)  0.232848   
STAEformer_subway_in_subway_out_calendar_early_...    35.88 (0.8\%)  0.133729   
STAEformer_subway_in_subway_out_calendar_early_...   35.95 (0.98\%)  0.174882   

                                                             mae_h1            \
                                                               mean       std   
STAEformer_subway_in_subway_out_calendar_early_...  20.36 (-0.17\%)  0.033285   
STAEformer_subway_in_subway_out_calendar_late_f...  20.37 (-0.11\%)  0.045057   
STAEformer_subway_in_calendar__e80_h1                 20.39 (0.0\%)  0.154526   
STAEformer_subway_in_subway_out_calendar_early_...  20.38 (-0.08\%)  0.137855   
STAEformer_subway_in_subway_out_calendar_late_f...   20.49 (0.49\%)  0.160206   
STAEformer_subway_in_subway_out_calendar_early_...   20.55 (0.78\%)  0.100583   
STAEformer_subway_in_subway_out_calendar_early_...   20.61 (1.04\%)  0.088788   

                                                           mase_h1            
                                                              mean       std  
STAEformer_subway_in_subway_out_calendar_early_...  0.61 (-0.17\%)  0.000994  
STAEformer_subway_in_subway_out_calendar_late_f...  0.61 (-0.11\%)  0.001350  
STAEformer_subway_in_calendar__e80_h1                 0.61 (0.0\%)  0.004623  
STAEformer_subway_in_subway_out_calendar_early_...  0.61 (-0.07\%)  0.004132  
STAEformer_subway_in_subway_out_calendar_late_f...   0.61 (0.49\%)  0.004799  
STAEformer_subway_in_subway_out_calendar_early_...   0.62 (0.78\%)  0.003015  
STAEformer_subway_in_subway_out_calendar_early_...   0.62 (1.04\%)  0.002652

rmse_h4            \
                                                               mean       std   
STAEformer_subway_in_subway_out_calendar_late_f...  44.09 (-1.23\%)  0.465219   
STAEformer_subway_in_subway_out_calendar_late_f...  44.17 (-1.05\%)  0.444862   
STAEformer_subway_in_subway_out_calendar_early_...  44.22 (-0.93\%)  0.452206   
STAEformer_subway_in_subway_out_calendar_early_...  44.57 (-0.16\%)  0.653885   
STAEformer_subway_in_calendar__e80_h4                 44.64 (0.0\%)  0.749028   
STAEformer_subway_in_subway_out_calendar_early_...    44.9 (0.58\%)  0.355340   
STAEformer_subway_in_subway_out_calendar_early_...   44.97 (0.76\%)  0.850792   

                                                             mae_h4            \
                                                               mean       std   
STAEformer_subway_in_subway_out_calendar_late_f...  23.79 (-1.05\%)  0.297099   
STAEformer_subway_in_subway_out_calendar_late_f...  23.86 (-0.76\%)  0.273204   
STAEformer_subway_in_subway_out_calendar_early_...  23.84 (-0.82\%)  0.275521   
STAEformer_subway_in_subway_out_calendar_early_...  23.98 (-0.25\%)  0.399274   
STAEformer_subway_in_calendar__e80_h4                 24.04 (0.0\%)  0.235967   
STAEformer_subway_in_subway_out_calendar_early_...    24.1 (0.27\%)  0.156948   
STAEformer_subway_in_subway_out_calendar_early_...    24.18 (0.6\%)  0.423145   

                                                           mase_h4            
                                                              mean       std  
STAEformer_subway_in_subway_out_calendar_late_f...  0.71 (-1.05\%)  0.008906  
STAEformer_subway_in_subway_out_calendar_late_f...  0.71 (-0.76\%)  0.008177  
STAEformer_subway_in_subway_out_calendar_early_...  0.71 (-0.82\%)  0.008238  
STAEformer_subway_in_subway_out_calendar_early_...  0.72 (-0.25\%)  0.011965  
STAEformer_subway_in_calendar__e80_h4                 0.72 (0.0\%)  0.007073  
STAEformer_subway_in_subway_out_calendar_early_...   0.72 (0.27\%)  0.004703  
STAEformer_subway_in_subway_out_calendar_early_...    0.72 (0.6\%)  0.012667


------------------ Exp1_subway_out ---


rmse_h1            \
                                                               mean       std   
STAEformer_subway_out_subway_in_calendar_late_f...  35.38 (-3.88\%)  0.173060   
STAEformer_subway_out_subway_in_calendar_early_...  35.71 (-2.99\%)  0.204204   
STAEformer_subway_out_subway_in_calendar_early_...  35.81 (-2.73\%)  0.340023   
STAEformer_subway_out_subway_in_calendar_early_...  35.84 (-2.63\%)  0.157459   
STAEformer_subway_out_subway_in_calendar_late_f...  35.87 (-2.55\%)  0.169968   
STAEformer_subway_out_subway_in_calendar_early_...  36.05 (-2.06\%)  0.278188   
STAEformer_subway_out_calendar__e80_h1                36.81 (0.0\%)  0.160002   

                                                             mae_h1            \
                                                               mean       std   
STAEformer_subway_out_subway_in_calendar_late_f...  19.68 (-3.51\%)  0.111560   
STAEformer_subway_out_subway_in_calendar_early_...  19.64 (-3.68\%)  0.088888   
STAEformer_subway_out_subway_in_calendar_early_...  19.76 (-3.11\%)  0.110338   
STAEformer_subway_out_subway_in_calendar_early_...   19.8 (-2.93\%)  0.060690   
STAEformer_subway_out_subway_in_calendar_late_f...   20.1 (-1.42\%)  0.067312   
STAEformer_subway_out_subway_in_calendar_early_...  19.74 (-3.22\%)  0.069391   
STAEformer_subway_out_calendar__e80_h1                20.39 (0.0\%)  0.082622   

                                                           mase_h1            
                                                              mean       std  
STAEformer_subway_out_subway_in_calendar_late_f...  0.61 (-3.06\%)  0.003469  
STAEformer_subway_out_subway_in_calendar_early_...  0.61 (-3.23\%)  0.002769  
STAEformer_subway_out_subway_in_calendar_early_...  0.61 (-2.65\%)  0.003437  
STAEformer_subway_out_subway_in_calendar_early_...  0.62 (-2.47\%)  0.001888  
STAEformer_subway_out_subway_in_calendar_late_f...  0.63 (-0.96\%)  0.002089  
STAEformer_subway_out_subway_in_calendar_early_...  0.61 (-2.77\%)  0.002160  
STAEformer_subway_out_calendar__e80_h1                0.63 (0.0\%)  0.002557

rmse_h4            \
                                                               mean       std   
STAEformer_subway_out_subway_in_calendar_early_...  40.59 (-4.14\%)  0.565579   
STAEformer_subway_out_subway_in_calendar_early_...  40.81 (-3.63\%)  0.313657   
STAEformer_subway_out_subway_in_calendar_early_...  40.88 (-3.45\%)  0.292779   
STAEformer_subway_out_subway_in_calendar_early_...  41.11 (-2.91\%)  0.184949   
STAEformer_subway_out_subway_in_calendar_late_f...  41.88 (-1.09\%)  0.500659   
STAEformer_subway_out_subway_in_calendar_late_f...  41.89 (-1.06\%)  0.148227   
STAEformer_subway_out_calendar__e80_h4                42.34 (0.0\%)  0.367245   

                                                             mae_h4            \
                                                               mean       std   
STAEformer_subway_out_subway_in_calendar_early_...   22.18 (-3.7\%)  0.180961   
STAEformer_subway_out_subway_in_calendar_early_...  22.55 (-2.08\%)  0.218737   
STAEformer_subway_out_subway_in_calendar_early_...  22.29 (-3.21\%)  0.155751   
STAEformer_subway_out_subway_in_calendar_early_...  22.55 (-2.06\%)  0.119015   
STAEformer_subway_out_subway_in_calendar_late_f...  22.85 (-0.75\%)  0.205196   
STAEformer_subway_out_subway_in_calendar_late_f...  22.95 (-0.32\%)  0.108604   
STAEformer_subway_out_calendar__e80_h4                23.03 (0.0\%)  0.209461   

                                                           mase_h4            
                                                              mean       std  
STAEformer_subway_out_subway_in_calendar_early_...   0.69 (-3.2\%)  0.005634  
STAEformer_subway_out_subway_in_calendar_early_...   0.7 (-1.58\%)  0.006807  
STAEformer_subway_out_subway_in_calendar_early_...  0.69 (-2.71\%)  0.004846  
STAEformer_subway_out_subway_in_calendar_early_...   0.7 (-1.55\%)  0.003705  
STAEformer_subway_out_subway_in_calendar_late_f...  0.71 (-0.24\%)  0.006391  
STAEformer_subway_out_subway_in_calendar_late_f...   0.71 (0.19\%)  0.003380  
STAEformer_subway_out_calendar__e80_h4                0.71 (0.0\%)  0.006488


------------------ Exp1 ---


rmse_h4            \
                                                               mean       std   
STAEformer_subway_out_subway_in_calendar_early_...   40.46 (-5.3\%)  0.182354   
STAEformer_subway_out_subway_in_calendar_early_...  40.76 (-4.61\%)  0.417094   
STAEformer_subway_out_subway_in_calendar_late_f...   40.8 (-4.52\%)  0.219264   
STAEformer_subway_out_subway_in_calendar_early_...  40.91 (-4.25\%)  0.391386   
STAEformer_subway_out_subway_in_calendar_late_f...  40.99 (-4.07\%)  0.358183   
STAEformer_subway_out_subway_in_calendar_late_f...  41.72 (-2.36\%)  0.231649   
STAEformer_subway_out_calendar__e200_h4               42.73 (0.0\%)  0.390762   

                                                             mae_h4            \
                                                               mean       std   
STAEformer_subway_out_subway_in_calendar_early_...  22.27 (-4.02\%)  0.095303   
STAEformer_subway_out_subway_in_calendar_early_...   22.3 (-3.88\%)  0.204857   
STAEformer_subway_out_subway_in_calendar_late_f...  22.51 (-2.96\%)  0.174603   
STAEformer_subway_out_subway_in_calendar_early_...  22.51 (-2.97\%)  0.085961   
STAEformer_subway_out_subway_in_calendar_late_f...  22.51 (-2.98\%)  0.215015   
STAEformer_subway_out_subway_in_calendar_late_f...  22.85 (-1.53\%)  0.155380   
STAEformer_subway_out_calendar__e200_h4                23.2 (0.0\%)  0.162759   

                                                           mase_h4            
                                                              mean       std  
STAEformer_subway_out_subway_in_calendar_early_...  0.69 (-3.53\%)  0.002965  
STAEformer_subway_out_subway_in_calendar_early_...  0.69 (-3.38\%)  0.006375  
STAEformer_subway_out_subway_in_calendar_late_f...   0.7 (-2.46\%)  0.005442  
STAEformer_subway_out_subway_in_calendar_early_...   0.7 (-2.47\%)  0.002674  
STAEformer_subway_out_subway_in_calendar_late_f...   0.7 (-2.48\%)  0.006684  
STAEformer_subway_out_subway_in_calendar_late_f...  0.71 (-1.02\%)  0.004837  
STAEformer_subway_out_calendar__e200_h4               0.72 (0.0\%)  0.005040

,rmse,mae,mase,legend_group,id,legend_group_str
STAEformer_subway_out_subway_in_calendar_late_fusion_traffic_model_backbone_InEmb24_adp16__e80_h1_bis1,35.361759,19.590174,0.609575,Late Fusion Other Methods,L_traffic_model_BB_In24_adp16,Late Fusion Other Methods
STAEformer_subway_out_subway_in_calendar_late_fusion_traffic_model_backbone_InEmb24_adp16__e80_h1_bis2,35.100483,19.538504,0.607977,Late Fusion Other Methods,L_traffic_model_BB_In24_adp16,Late Fusion Other Methods
STAEformer_subway_out_subway_in_calendar_late_fusion_traffic_model_backbone_InEmb24_adp16__e80_h1_bis3,35.537338,19.767555,0.615094,Late Fusion Other Methods,L_traffic_model_BB_In24_adp16,Late Fusion Other Methods
STAEformer_subway_out_subway_in_calendar_late_fusion_traffic_model_backbone_InEmb24_adp16__e80_h1_bis4,35.507175,19.688236,0.612629,Late Fusion Other Methods,L_traffic_model_BB_In24_adp16,Late Fusion Other Methods
STAEformer_subway_out_subway_in_calendar_late_fusion_traffic_model_backbone_InEmb24_adp16__e80_h1_bis5,35.400440,19.798395,0.616056,Late Fusion Other Methods,L_traffic_model_BB_In24_adp16,Late Fusion Other Methods
STAEformer_subway_out_calendar__e80_h1_bis1,36.859035,20.458881,0.633650,Baseline,Baseline,Baseline
STAEformer_subway_out_calendar__e80_h1_bis2,36.969063,20.501513,0.634974,Baseline,Baseline,Baseline
STAEformer_subway_out_calendar__e80_h1_bis3,36.931046,20.311161,0.629079,Baseline,Baseline,Baseline
STAEformer_subway_out_calendar__e80_h1_bis4,36.692570,20.355349,0.630452,Baseline,Baseline,Baseline
STAEformer_subway_out_calendar__e80_h1_bis5,36.594925,20.337545,0.629897,Baseline,Baseline,Baseline


In [7]:
# experiences = ['Exp1_subway_in','Exp1_subway_out','Exp1']
exp_i = 'Exp1_subway_in' # 'Exp1_subway_out' #'Exp1_subway_in' # 'Exp1' 
metrics = ['rmse','mae','mase']
trials = dic_trials[exp_i]
print(f"\n------------------ {exp_i} ---")

L_metrics = []
for trial_j in trials:
    L_metrics = tackle_trial_for_distrib(folder_path,dic_exp_to_names,L_metrics,exp_i,trial_j,metrics)
df_metrics_all = pd.concat(L_metrics)

def change_legend(row):
    if row.legend_group == 'Other Methods':
        if 'early_fusion' in row.name:
            return 'Early Fusion Other Methods '
        elif 'late_fusion' in row.name:
            return 'Late Fusion Other Methods '
        else:
            return row.legend_group
    else:
        return row.legend_group

def reduce_id(x):
    x = x.replace('late_fusion_','L_')
    x = x.replace('early_fusion_','E_')
    x = x.replace('backbone','BB')
    return x 


configs_to_keep = [
    'Baseline',
    # Model backbone : ---- 
    'late_fusion_traffic_model_backbone_In24_adp16',
    'early_fusion_traffic_model_backbone_In24_adp16',
    # CABB with adpQ = 0 ---
    'early_fusion_CABB_In24_ctxIn8_adp16_adpQ0',
    'late_fusion_CABB_In24_ctxIn8_adp16_adpQ0',
    # CABB with adpQ = 24 ---
    'early_fusion_CABB_In24_ctxIn8_adp16_adpQ24',
    'late_fusion_CABB_In24_ctxIn8_adp16_adpQ24',
    # 'early_fusion_CABB_In24_ctxIn24_adp16_adpQ24',
    # 'late_fusion_CABB_In24_ctxIn24_adp16_adpQ24',
    
    # Others ---
    'late_fusion_simple_embedding',
    'early_fusion_shared_embedding',
    'early_fusion_independant_embedding'   
] 
# configs_to_keep = None 

palette = ['#4e79a7','#f28e2b','#e15759', '#59a14f', '#c7e9c0','#1a4314' ]
legend_groups = ['Baseline','Independant Embedding','Shared Embedding','Early Fusion Other Methods ','Late Fusion Other Methods ']
horizons = list(set([c.split('_')[-1][1:] for c in df_metrics_all.columns]))
for horizon in horizons:
    print('\n----------------')
    print(f"Horizon: {horizon}")
    df_horizon = df_metrics_all[[c for c in df_metrics_all.columns if c.endswith(f"_h{horizon}")]].dropna()

    df_horizon = update_df_metrics(df_horizon,exp_i)
    df_horizon['legend_group'] = df_horizon.apply(change_legend,axis = 1)

    if configs_to_keep is not None: 
        df_horizon = df_horizon[df_horizon['id'].isin(configs_to_keep)]
        df_horizon['id'] = df_horizon['id'].apply(reduce_id)

    df_horizon['id'] = df_horizon['id'].apply(lambda x: x.replace('_In24_adp16',''))

    # plot_boxplot_on_metric(df_horizon, metric_i='mae', xaxis_label="Config", legend_group='legend_group', width=800, height=600, 
    #                             save_path=None,palette =palette ,legend_groups =legend_groups )
    title = f'RMSE Distribution per Config and Fusion Strategy of Subway-In Prediction at Horizon [{15*(int(horizon)-1)} - {15*(int(horizon))}]min'
    plot_boxplot_on_metric(df_horizon, metric_i='rmse', xaxis_label="Config", legend_group='legend_group', width=800, height=600, 
                                save_path=None,palette =palette ,legend_groups =legend_groups,title = title)
    # plot_boxplot_on_metric(df_horizon, metric_i='mase', xaxis_label="Config", legend_group='legend_group', width=1200, height=400, 
    #                             save_path=None)
    


------------------ Exp1_subway_in ---

----------------
Horizon: 1


Loading BokehJS ...


----------------
Horizon: 4


Loading BokehJS ...

In [9]:
# --- Captions Dictionary ---
captions = {
    "Exp1": r"Prediction of Subway-Out with STAEformer using contextual data 'Subway-In' according to different Integration Strategies. Metrics are averaged over 5 runs.",
    "Exp1_subway_in": r"Prediction of Subway-In with STAEformer using contextual data 'Subway-Out' according to different Integration Strategies. Metrics are averaged over 5 runs.",
    'Exp1_subway_out': r"Prediction of Subway-Out with STAEformer using contextual data 'Subway-In' according to different Integration Strategies. Metrics are averaged over 5 runs.",
}

# --- Generate and Print LaTeX Tables ---

print("\n------------------------------------------------\n%%% LaTeX code for Experiment 1 Horizon +15min Table %%%\n")
latex_table_1 = dataframe_to_latex(df_metrics_Exp1_subway_in_h1, captions["Exp1_subway_in"], "exp1_subway_context", lambda x: parse_index_exp1_2(x, contextual='subway_out'))
print(latex_table_1)

print("\n" + "="*80 + "\n")

print("\n------------------------------------------------\n%%% LaTeX code for Experiment 1 Horizon +60min Table %%%\n")
latex_table_1 = dataframe_to_latex(df_metrics_Exp1_subway_in_h4, captions["Exp1_subway_in"], "exp1_subway_context", lambda x: parse_index_exp1_2(x, contextual='subway_out'))
print(latex_table_1)

print("\n" + "="*80 + "\n")



print("\n------------------------------------------------\n%%% LaTeX code for Experiment 1 Horizon +15min Table %%%\n")
latex_table_1 = dataframe_to_latex(df_metrics_Exp1_subway_out_h1, captions["Exp1_subway_out"], "exp1_subway_context", lambda x: parse_index_exp1_2(x, contextual='subway_in'))
print(latex_table_1)

print("\n" + "="*80 + "\n")

print("\n------------------------------------------------\n%%% LaTeX code for Experiment 1 Horizon +60min Table %%%\n")
latex_table_1 = dataframe_to_latex(df_metrics_Exp1_subway_out_h4, captions["Exp1_subway_out"], "exp1_subway_context", lambda x: parse_index_exp1_2(x, contextual='subway_in'))
print(latex_table_1)

print("\n" + "="*80 + "\n")


print("\n------------------------------------------------\n%%% LaTeX code for Experiment ? Horizon +?min Table %%%\n")
print("%%% LaTeX code for Experiment 1 Table %%%\n")
latex_table_1 = dataframe_to_latex(df_metrics_Exp1_h4, captions["Exp1"], "exp1_subway_context", parse_index_exp1_2)
print(latex_table_1)

print("\n" + "="*80 + "\n")


------------------------------------------------
%%% LaTeX code for Experiment 1 Horizon +15min Table %%%

\begin{table}[!htb]
    \centering
    \caption{Prediction of Subway-In with STAEformer using contextual data 'Subway-Out' according to different Integration Strategies. Metrics are averaged over 5 runs.}
    \label{tab:exp1_subway_context}
    \resizebox{\textwidth}{!}{
    \begin{tabular}{{llcccccc}}
        \toprule
        Contextual Data & Integration Strategy & \multicolumn{2}{c}{ RMSE (h1) } & \multicolumn{2}{c}{ MAE (h1) } & \multicolumn{2}{c}{ MASE (h1) } \\
        \cmidrule(lr){1-2} \cmidrule(lr){3-4} \cmidrule(lr){5-6} \cmidrule(lr){7-8}
         &  & Mean & Std & Mean & Std & Mean & Std \\
        \midrule
        Yes & Early Fusion Shared Embedding & 35.48 (-0.33\%) & 0.1352 & 20.36 (-0.17\%) & 0.0333 & 0.61 (-0.17\%) & 0.0010 \\
        Yes & Late Fusion Traffic Model Backbone Inemb24 Adp16 & 35.58 (-0.04\%) & 0.0590 & 20.37 (-0.11\%) & 0.0451 & 0.61 (-0.11\%) & 0.

In [22]:

import os 
import sys 

current_path = os.path.abspath(os.getcwd())
parent_dir = os.path.abspath(os.path.join(current_path, '..','..','..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from examples.accuracy_comparison import load_trainer_ds_from_2_trials,get_predict_real_and_inputs,get_model_args
from examples.accuracy_comparison import get_rainy_indices,plot_analysis_comparison_2_config,get_previous_and_prediction,get_gain_from_mod1
import torch 
from pipeline.utils.metrics import evaluate_metrics

def get_dict_metrics_on_rainy_events(full_predict1,full_predict2,Y_true,X,args_init1,args_init2,ds2):
    h_idx = 1
    metric_list = ['rmse','mse','mae','mase','mape']
    previous,_,_,_ = get_previous_and_prediction(full_predict1,full_predict2,Y_true,X,h_idx)
    assert args_init1.horizon_step == args_init1.step_ahead, "Horizon step must be equal to step_ahead here"

    print("\nComparison on between models across all time-slots followed by comparison on Rainy Events Only")
    _,train_rainy_indices,_ = get_rainy_indices(args = args_init2,ds = ds2,training_mode = 'train')
    print(f"Number of rainy time-slots in the train set: {len(train_rainy_indices)}, i.e {len(train_rainy_indices)/len(ds2.tensor_limits_keeper.df_verif_train)*100:.2f} % of the train set")
    # ---- Plot Accuracy comparison on rainy moments only ----
    mask,rainy_indices,df_weather = get_rainy_indices(args = args_init2,ds = ds2,training_mode = 'test')
    print(f"Number of rainy time-slots in the test set: {len(rainy_indices)}, i.e {len(rainy_indices)/len(ds2.tensor_limits_keeper.df_verif_test)*100:.2f} % of the test set\n")

    dates = mask[mask].index
    masked_index = mask.reset_index(drop=True)
    masked_index = masked_index[masked_index].index


    # --- Get Prediction on rainy time-slots:
    rainy_predict1 = torch.index_select(full_predict1,0,torch.tensor(masked_index).long())
    rainy_predict2 = torch.index_select(full_predict2,0,torch.tensor(masked_index).long())
    rainy_Y_true = torch.index_select(Y_true,0,torch.tensor(masked_index).long())
    rainy_previous = torch.index_select(previous,0,torch.tensor(masked_index).long())

    dic_metric1 = evaluate_metrics(rainy_predict1,rainy_Y_true,metrics = metric_list, previous = rainy_previous,horizon_step = h_idx)
    dic_metric1 = dict(dic_metric1.copy())
    dic_metric2 = evaluate_metrics(rainy_predict2,rainy_Y_true,metrics = metric_list, previous = rainy_previous,horizon_step = h_idx)
    dic_metric2 = dict(dic_metric2.copy())
    return dic_metric1,dic_metric2



dic_contextual_data = {'bike_out': [[]],  #,['weather','calendar']
                    }

dic_fusion_strategie = {('bike_out',()): [''],
                        ('bike_out',('weather','calendar')):['early_fusion','late_fusion'],
                        }
             
dic_feature_extractor = {('bike_out',()): [''],
                        ('bike_out',('weather','calendar')):['repeat_t_proj','feature_extractor']
                        }


model_name = 'STAEformer'
epochs = 200
range_k = range(1,6)
training_mode = 'test'
modification = {'shuffle':False,
                'data_augmentation':False,
                'torch_compile': False,
                }
log = ''
Exp = 'Exp2'  # 'Exp2'

for horizon in [4]:
    for target in ['bike_out']:
        for contextual_data in dic_contextual_data[target]:
            reversed_metric = False
            fusion_strategies = dic_fusion_strategie[(target,tuple(contextual_data))]
            for fusion_strategie in fusion_strategies:
                feature_extractors = dic_feature_extractor[(target,tuple(contextual_data))]
                for feature_extractor in feature_extractors:
                    if not('weather' in contextual_data):
                        print('\n------------------------------------------------------\nContextual data:\n')
                        contextual_data1 = ['weather','calendar'] if target == 'bike_out' else ['subway_in','weather','calendar']
                        fusion_strategie1 = dic_fusion_strategie[(target,tuple(contextual_data1))][0]
                        feature_extractor1 = dic_feature_extractor[(target,tuple(contextual_data1))][0]

                        trial_id1 = f"{model_name}_{target}_calendar__e{epochs}_h{horizon}_bis"
                        trial_id2 = f"{model_name}_{target}_{'_'.join(contextual_data1)}_{fusion_strategie1}_{feature_extractor1}__e{epochs}_h{horizon}_bis"
                        reversed_metric = True 
                    else:
                        trial_id1 = f"{model_name}_{target}_calendar__e{epochs}_h{horizon}_bis"
                        trial_id2 = f"{model_name}_{target}_{'_'.join(contextual_data)}_{fusion_strategie}_{feature_extractor}__e{epochs}_h{horizon}_bis"

                    model_args,_,path_model_args,_ = get_model_args(target,model_name,save_folder_name = f'{Exp}/{target}_{model_name}')
                    ds1,ds2,args_init1,args_init2 = None, None, None, None
                    for k in range_k:
                        trial_id1_updated = f"{trial_id1}{k}_f5"
                        trial_id2_updated = f"{trial_id2}{k}_f5"

                        trainer1,trainer2,ds1,ds2,args_init1,args_init2 = load_trainer_ds_from_2_trials(trial_id1_updated,trial_id2_updated,modification = modification,
                                                                                                        model_args=model_args,
                                                                                                        path_model_args=path_model_args,
                                                                                                        ds1_init=ds1,ds2_init=ds2,
                                                                                                        args_init1=args_init1,args_init2=args_init2,
                                                                                                        )
                                                                                                        

                        full_predict1,full_predict2,Y_true,X = get_predict_real_and_inputs(trainer1,trainer2,ds1,ds2,training_mode=training_mode)

                        globals()[f"trainer1_bis{k}"] = trainer1
                        globals()[f"trainer2_bis{k}"] = trainer2
                        globals()[f"ds1_bis{k}"] = ds1
                        globals()[f"ds2_bis{k}"] = ds2
                        globals()[f"full_predict1_bis{k}"] = full_predict1
                        globals()[f"full_predict2_bis{k}"] = full_predict2

                        dic_metric1,dic_metric2 = get_dict_metrics_on_rainy_events(globals()[f"full_predict1_bis{k}"],globals()[f"full_predict2_bis{k}"],Y_true,X,args_init1,args_init2,ds2)

                        # Keep track on metric from model 1
                        if reversed_metric:
                            dic_metric2 = dict(dic_metric1.copy())
                        RMSE = dic_metric2['rmse_all']
                        MAE = dic_metric2['mae_all']
                        MASE = dic_metric2['mase_all']
                        MAPE = dic_metric2['mape_all']
                        if feature_extractor == []:
                            log += f"{model_name}_{target}__e{epochs}_h{horizon}_bis{k}:   All Steps RMSE = {RMSE:.3f}, MAE = {MAE:.3f}, MASE = {MASE:.3f}, MAPE = {MAPE:.3f}\n"
                        else:
                            log += f"{model_name}_{target}_{'_'.join(contextual_data)}_{fusion_strategie}_{feature_extractor}__e{epochs}_h{horizon}_bis{k}:   All Steps RMSE = {RMSE:.3f}, MAE = {MAE:.3f}, MASE = {MASE:.3f}, MAPE = {MAPE:.3f}\n"

                    print(log)


------------------------------------------------------
Contextual data:

Trial ID 1:  STAEformer_bike_out_calendar__e200_h4_bis1_f5
Trial ID 2:  STAEformer_bike_out_weather_calendar_early_fusion_repeat_t_proj__e200_h4_bis1_f5
model_save_path for trial id1:  /home/rrochas/prediction-validation/save/K_fold_validation/training_wo_HP_tuning/Exp2/bike_out_STAEformer/best_models/STAEformer_bike_out_calendar__e200_h4_bis1_f5.pkl
----------------------------------------
Loading the Complete Dataset for K-fold splitting
Coverage Period: 35040 elts between 2019-01-01 00:00:00 and 2019-12-31 23:45:00
Invalid dates within this fold: 0

>>>Tackle Target dataset: bike_out
Loading from /home/rrochas/../../data/rrochas/prediction_validation/agg_data/velov/velov_emitted_by_station15min.csv...
df pivoted:  (70049, 434)
df reindexed :  (35040, 434)
Len coverage period:  35040
df filtered:  (35040, 434)
   Loaded data: (35040, 434)
   Dimension after spatial agg: (35040, 26)
   Init Dataset: 'torch.Size(